In [149]:
# import packages
from bs4 import BeautifulSoup as bs
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import time
from config import username, password
import datetime
import pandas as pd
from selenium import webdriver
import requests
from webdriver_manager.chrome import ChromeDriverManager
from collections import defaultdict

In [56]:
# init browser
executable_path = {'executable_path': '/Users/yangsun/Downloads/chromedriver 5'}
browser = Browser('chrome', **executable_path, headless=False)

In [57]:
# visit url
login_url='https://www.glassdoor.com/profile/login_input.htm'
browser.visit(login_url)

In [58]:
# sign in
browser.find_by_id('inlineUserEmail').fill(username)
browser.find_by_id('inlineUserPassword').fill(password)
browser.find_by_xpath('//button[@type="submit"]').click()

In [59]:
# go to new url
url='https://www.glassdoor.com/Salary/Panda-Express-Salaries-E5659.htm'
browser.visit(url)

In [213]:
def select_area(area_name):
    """select area from selector
    """
    browser.find_by_xpath('//button[@class="gd-ui-button d-none d-lg-inline-block css-pitbid"]').click()
    browser.find_by_xpath('//span[@class="SVGInline arrowDown"]')[1].click()
    browser.find_by_text(area_name).first.click()

In [220]:
area_name='- Los Angeles, CA Area'
select_area(area_name)

In [210]:
html=browser.html
soup = bs(html, 'html.parser')
# list of locations from 23rd starting
soup.find_all('li',class_='dropdownOption')[23]

In [247]:
# all pay list
pay_lists=soup.find_all('div',class_='css-1u4lhyp')

In [215]:
# only look for 
title_list = ['general manager',
              'assistant manager',
              'counter help',
              'counter helper',
              'shift leader',
              'shift lead',
              'kitchen help',
              'kitchen helper',
              'lead counter',
              'lead counter help',
              'front of house',
              'back of house',
              'cashier',
              'crew member',
              'chef',
              'pic',
              'FOH team member',
              'kitchen staff',
              'FOH']

In [246]:
def get_frontend_info(pay_list):
    """Extract front end job title information
           - front end title
           - avg total pay
           - additional pay
       Return: 
           {front end title:[avg base pay, base pay, additional pay]}
    """
    # front_end_info= {front end title:[avg base pay, base pay, additional pay]}
    front_end_info=defaultdict(list)
    title=pay_list.a.strong.text
    # pay
    pay=pay_list.span.strong.text
    # additional pay
    tag=pay_list.find_all('span',text='Additional Pay')
    # check if additional pay tag exist
    if len(tag)==0:
        additional_pay=0
    additional_pay=tag[0].find_parent('div').strong.text
    front_end_info[title]=[pay,additional_pay]
    return front_end_info

In [ ]:

for pay_list in pay_lists:
    title=pay_list.a.strong.text
    if title not in title_list:
        continue
    # get front data
    
    pay_list.span.strong.text
    sub_url=url+pay_list[0].a['href']
    # go to sub_url
    browser.visit(sub_url)
    sub_html=browser.html
    

In [237]:
tag=pay_list[0].find_all('span',text='Additional Pay')
tag[0].find_parent('div').strong.text

'$0'

In [216]:
# find job title
pay_list[0].a.strong.text

'Counter Help'

In [217]:
# find pay
pay_list[0].span.strong.text

'$14'

In [218]:
# sub_url
sub_url=url+pay_list[0].a['href']

In [219]:
# go to sub_url
browser.visit(sub_url)
sub_html=browser.html

In [112]:
# based pay average and additional pay
soup_sub=bs(sub_html,'html.parser')
avg_base_pay=soup_sub.find_all('div',class_='col-12 col-lg-4')[0].h2.text
additional_pay=soup_sub.find_all('div',class_='col-12 col-lg-7 d-flex align-items-baseline')[0].h2.text

'$29,400'

In [121]:
# compare with market
market_comparison=soup_sub.find_all('strong',class_='css-1uyte9r css-1qxtz39 css-fkag2m css-ztwr07 eb8fmw1')[0].text

In [136]:
# related salaries with others
text='Related Salaries'
tag=soup_sub.find("h3", text=text)
container=tag.find_parent('div')

In [148]:
# container information
s_c=container.find_all('div',class_='row no-gutters mx-0 py align-items-center css-1u4lhyp')
title=s_c[0].p.a.text
name=s_c[0].find_all('p',class_='m-0 strong')[0].text
pay=s_c[0].find_all('strong',class_='d-block css-1djk842')[0].text

In [151]:
# get all container information
# company name, title, pay
# related company={company name:[title, pay]}
related_company=defaultdict(list)
for sub in s_c:
    # pay title
    title=sub.p.a.text
    # company name
    name=sub.find_all('p',class_='m-0 strong')[0].text
    # pay
    pay=sub.find_all('strong',class_='d-block css-1djk842')[0].text
    # append to list
    related_company[name]=[title, pay]
related_company

defaultdict(list,
            {"Jersey Mike's": ['Counter Help - Hourly', '$16 - $17'],
             'Hanmi Financial': ['Counter Help', '$44K - $47K'],
             'Walters Wholesale Electric': ['Counter Help - Hourly',
              '$20 - $22']})

In [154]:
# go back to previous url
browser.visit(url)